# Task 1.1  Pauli Operators

## Objective 1:  Define Pauli Operators

Pauli operators are fundamental building blocks in quantum computing that represent single-qubit and multi-qubit operations. In Qiskit, the `Pauli` class provides various ways to create and manipulate these operators.

You can find more information about Pauli Operators here
https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.Pauli 

### Initialization Methods

#### 1. Pauli String Representation

Pauli operators can be created from string representations using the Pauli matrices:
- `I`: Identity operator
- `X`: Pauli-X (bit-flip) operator
- `Y`: Pauli-Y operator
- `Z`: Pauli-Z (phase-flip) operator

Note: Qiskit uses <b>Little Endian Notation</b> where qubit(n-1) is the leftmost and qubit0 is the rightmost in the string representation.

The phase can be specified as ` ` (default), `-`, `i`, or `-i`.

In [83]:
from qiskit.quantum_info import Pauli

# Create Pauli operators from string representations
p = Pauli('XZ')      # No phase (default +1)
p2 = Pauli('iYY')    # With imaginary phase i
p3 = Pauli('-iZX')   # With negative imaginary phase -i

print(f"P:{p}, P2:{p2}, P3:{p3}")

P:XZ, P2:YY, P3:-iZX


#### 2. Boolean Array Representation

Pauli operators can also be created using boolean numpy arrays:
- `z` array: Specifies Z components (True = Z operator, False = I operator)
- `x` array: Specifies X components (True = X operator, False = I operator)

The actual Pauli operator at each qubit position is determined by:
- (z=False, x=False) → I (Identity)
- (z=False, x=True) → X
- (z=True, x=False) → Z
- (z=True, x=True) → Y (since Y = iXZ)

In [84]:
import numpy as np

# Define Z and X components using boolean arrays
# I on qubit 0, X on qubit 1, Z on qubit 2, Y on qubit 3
z = np.array([False,False,True,True])  
x = np.array([False,True,False,True])  

pauli_array_rep = Pauli((z, x))

print(f"Pauli Operator from Boolean Array: {pauli_array_rep}")

Pauli Operator from Boolean Array: YZXI


#### 3. Quantum Circuit Representation

Pauli operators can be extracted from quantum circuits that contain only Pauli gates (I,X, Y, Z). The circuit is analyzed to construct the corresponding Pauli operator.

In [85]:
from qiskit import QuantumCircuit

# Create a quantum circuit with Pauli gates
qc = QuantumCircuit(4)
qc.id(0) # I gate on qubit 0 Not Needed just usedfor demonastration
qc.z(1)  # Z gate on qubit 1
qc.x(2)  # X gate on qubit 2
qc.y(3)  # Y gate on qubit 3
# No Other Gates should be used , uncommenting the line below will return an error
#qc.cx(0,1)


# Extract Pauli operator from the circuit
pauli_quantum_circuit = Pauli(qc)

print(f"Pauli Operator from quantum circuit : {pauli_quantum_circuit}")

Pauli Operator from quantum circuit : YXZI


#### 4. From ScalarOp

Pauli operators can be combined with scalar operations. The `ScalarOp` represents a scalar multiple of the identity operator, which can be composed with a Pauli operator to apply a global phase.

In [90]:
from qiskit.quantum_info import Pauli, ScalarOp

# Create a scalar operator (scaled identity)
# Pauli can only be multiplied by 1, -1j, -1, 1j
scalar_op = ScalarOp(dims=(2,2,2,2), coeff=-1j)  # -1*j

# Create a Pauli operator
pauli_scalarop = Pauli('YXZI')

# Compose the scalar operator with Pauli operator
# This applies the scalar coefficient to the Pauli operator
p = scalar_op.compose(pauli_scalarop)

print(f"Pauli Operator Before applying Scalar : {pauli_scalarop}")

print(f"Pauli Operator from ScalarOp : {p}")

Pauli Operator Before applying Scalar : YXZI
Pauli Operator from ScalarOp : -iYXZI


### Representation Methods

#### 1. Matrix Representation

The `to_matrix()` method converts a Pauli operator to its matrix representation. For multi-qubit Pauli operators, this returns the tensor product of individual Pauli matrices.

In [91]:
# Convert Pauli operator to matrix representation
p = Pauli('ZX')
p_matrix = p.to_matrix()

print(f"Matrix Format: {p_matrix}")

[[ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]]


#### 2. String Representation

Pauli operators can be converted to string labels for easy readability. The `to_label()` method provides a canonical string representation.

In [92]:
# String representations
print(f"String format:{str(p)}")

print(f"Label format:{p.to_label()}")

String format:ZX
Label format:ZX


### Attributes

The Pauli class provides several attributes to access properties of the operator, some of them are:
- `dim`: Dimension of the operator
- `num_qubits`: Number of qubits
- `num_clbits`: Number of classical bits (always 0 for Pauli operators)
- `phase`: Phase of the operator (0, 1, 2, 3 representing +1, i, -1, -i)
- `x`: X component as boolean array
- `z`: Z component as boolean array

In [93]:
# Access Pauli operator attributes
print(f"Dimension: {p.dim}")  # Dimension of the operator matrix
print(f"Classical bits: {p.num_clbits}, Qubits: {p.num_qubits}")  # Number of qubits
print(f"X: {p.x}, Z: {p.z}, Phase: {p.phase}")  # Components and phase

Dimension: (4, 4)
Classical bits: 0, Qubits: 2
X: [ True False], Z: [False  True], Phase: 0


### Methods

The Pauli class provides various methods for operator manipulation and analysis. some of them are

#### adjoint

Returns the adjoint (conjugate transpose) of the Pauli operator. For Pauli operators, the adjoint is the same as the operator itself (they are Hermitian), but the phase may change.

In [94]:
# Get the adjoint (conjugate transpose) of the Pauli operator
p.adjoint()
print(f"pauli operator:{p3}, adjoint: {p3.adjoint()}")

pauli operator:-iZX, adjoint: iZX


#### anticommutes

Checks if two Pauli operators anticommute. Two operators A and B anticommute if AB = -BA.

In [95]:
# Check if two Pauli operators anticommute
p = Pauli('X')      # X
p2 = Pauli('Y')     # Y
print(f"{p} anticommutes with {p2} is {p.anticommutes(p2)}")

X anticommutes with Y is True


#### compose

Returns the composition (matrix product) of two Pauli operators. The composition of Pauli operators results in another Pauli operator (up to a phase).

In [96]:
# Compose two Pauli operators (matrix multiplication)

print(f"{p} compose with {p.adjoint()} is {p.compose(p.adjoint())}")

X compose with X is I


#### conjugate

Returns the complex conjugate of the Pauli operator. For real Pauli operators (without imaginary phases), this is the same as the original operator.

In [97]:
# Get the complex conjugate of the Pauli operator
p = Pauli('ZX')
print(f"Conjugate of {p} is {p.conjugate()}")
print(f"Conjugate of {p2} is {p2.conjugate()}")

Conjugate of ZX is ZX
Conjugate of Y is -Y


#### delete

Returns a Pauli operator with specified qubits deleted. This reduces the number of qubits in the operator.

In [78]:
# Delete a qubit from the Pauli operator
delete_qubit = 0
delete_0 = p.delete(delete_qubit)

print(f"{p} after deleting qubit {delete_qubit} is {delete_0}")

ZX after deleting qubit 0 is Z


#### insert

Returns a Pauli operator with additional qubits inserted at specified positions.

In [80]:
# Insert a Pauli operator at a specific qubit position
insert_0 = delete_0.insert(delete_qubit, Pauli('X'))

print(f"{delete_0} after inserting X at qubit {delete_qubit} is {insert_0}")

Z after inserting X at qubit 0 is ZX


#### to_instruction

Converts the Pauli operator to a quantum instruction that can be used in quantum circuits.

In [81]:
# Convert Pauli operator to a quantum instruction
p.to_instruction()

Instruction(name='pauli', num_qubits=2, num_clbits=0, params=['ZX'])